In [1]:
!pip install transformers[torch] datasets evaluate seqeval bitsandbytes accelerate peft trl
!pip install torch
!pip install --upgrade huggingface_hub
!pip install tqdm
!pip install sentencepiece
!pip install protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 40.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 k

In [2]:
!pip uninstall -y pillow
!pip install pillow==9.4.0

Found existing installation: Pillow 9.3.0
Uninstalling Pillow-9.3.0:
  Successfully uninstalled Pillow-9.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 17.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
from huggingface_hub import login
from utils.vlm_model import EndpointHandler
import torch

login(token="FILL IN")
handler = EndpointHandler(
    base_model_id="llava-hf/llava-v1.6-34b-hf",#"llava-hf/llava-v1.6-mistral-7b-hf",
    adapter_model_id="ekolasky/llava-v1.6-34b-email-entities",#"ekolasky/llava-v1.6-mistral-email-entities", 
    # load_in_8bit=True
    load_in_4bit=True
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [2]:
from datasets import load_dataset, Dataset

# Load val dataset
val_dataset = load_dataset("ekolasky/EntityExtractionFromEmails", split="val")

# Get evals for whole val set

In [5]:
evals = []
for index, example in enumerate(val_dataset):
    print(index)
    outputs = handler({
        'inputs': {
            'image': example['image']
        }
    })
    predictions = outputs['entities']
    errors = outputs['errors']

    # Reformat labels
    labels = []
    entities = example['entities']
    for i in range(len(entities['category'])):
        labels.append({
            'name': entities['name'][i],
            'category': entities['category'][i]
        })

    # Add to evals
    evals.append({
        'index': index,
        'labels': labels,
        'predictions': predictions,
        'errors': errors
    })

0
-
Name
:
Silicon
Coul
oir
|
Category
:
company
<|im_end|>
1
-
Name
:
Nat
Eli
ason
|
Category
:
person


-
Name
:
Traditional
Publishing
is
Great
,
Actually
|
Category
:
company
<|im_end|>
2
-
Name
:
Wyoming
Ball
oon
Company
|
Category
:
company


-
Name
:
B
ret
King
|
Category
:
person
<|im_end|>
3
-
Name
:
Mary
let
tic
oy
ne
|
Category
:
person
<|im_end|>
4
-
Name
:
Ethan
Kol
ask
y
|
Category
:
person


-
Name
:
Nat
asha
Mon
roy
|
Category
:
person
<|im_end|>
5
-
Name
:
Ethan
|
Category
:
person


-
Name
:
Tech
stars
|
Category
:
company
<|im_end|>
6
-
Name
:
Britt
ney
Z
ie
bell
|
Category
:
person


-
Name
:
G
ail
ie
Gordon
|
Category
:
person
<|im_end|>
7
-
Name
:
Caroline
Craw
ley
|
Category
:
person
<|im_end|>
8
-
Name
:
Molly
Tem
pl
in
|
Category
:
person


-
Name
:
The
Pack
ing
Sign
Up

2
0
2
4
|
Category
:
company
<|im_end|>
9
-
Name
:
William
Kol
ask
y
|
Category
:
person


-
Name
:
Hughes
Hubbard
&
Reed
L
LP
|
Category
:
company
<|im_end|>


In [6]:
# Save evals to json
import json

with open('./evals/eval5.json', 'w') as json_file:
    json.dump(evals, json_file)

# Test Single Example

In [4]:
from PIL import Image

example = val_dataset[4]
def stream_callback(entity):
    print(entity)

_ = handler({'inputs': {'image': example['image']}}, stream_callback=stream_callback)


{'name': 'Ethan Kolasky', 'category': 'person'}
{'name': 'Natasha Monroy', 'category': 'person'}


In [25]:
print(handler.processor.tokenizer.encode('\n', add_special_tokens=False))

[59568, 144]


In [4]:
handler.processor.tokenizer.eos_token_id

7

In [6]:
def split_list(input_list, split_element):
    result = []
    current_part = []
    for item in input_list:
        if item == split_element:
            if current_part:
                result.append(current_part)
                current_part = []
        else:
            current_part.append(item)
    
    result.append(current_part)
    return result

In [10]:
split_list([59594, 10636, 59601, 42940, 14330, 4099, 7876, 1808, 33784, 59601, 2165, 144, 52], handler.processor.tokenizer.encode('\n')[1])

[[59594, 10636, 59601, 42940, 14330, 4099, 7876, 1808, 33784, 59601, 2165],
 [52]]

# Evaluate Existing Eval File

In [7]:
import json
with open('./evals/eval5.json', 'r', encoding='utf-8') as file:
    example_evals = json.load(file)

In [8]:
from evaluator import Evaluator

evaluator = Evaluator(example_evals)

{'name': 'General Catalyst', 'category': 'company'}
{'name': 'MultiOn', 'category': 'company'}
{'name': 'Agnetic', 'category': 'company'}
{'name': 'IBM', 'category': 'company'}
{'name': 'Microsoft', 'category': 'company'}
{'name': 'Techstars Accelerator', 'category': 'company'}
{'name': 'Gaile Gordon', 'category': 'person'}
{'name': 'Hughes Hubbard & Reed', 'category': 'company'}
{'percent matching results': 0.5, 'percent missing results': 0.4444444444444444, 'percent extra results': 0.3888888888888889, 'percent miscategorized results': 0.1, 'percent errors': 0.0}


In [17]:
evaluator.error_metrics

{'percent matching results': 0.6666666666666666,
 'percent missing results': 0.2777777777777778,
 'percent extra results': 0.2222222222222222,
 'percent miscategorized results': 0.07692307692307693,
 'percent errors': 0.0}

In [18]:
evaluator.view_results()

Example 0
Errors: []
Missing results: []
Extra results: []
Matching results: []
Miscategorized results: [[{'name': 'Silicon Couloir', 'category': 'company'}, {'name': 'Silicon Couloir', 'category': 'person'}]]

Example 1
Errors: []
Missing results: []
Extra results: ['Traditional Publishing is Great, Actually']
Matching results: [['Nat Eliason', 'Nat Eliason']]
Miscategorized results: []

Example 2
Errors: []
Missing results: []
Extra results: []
Matching results: [['Wyoming Balloon Company', 'Wyoming Balloon Company'], ['Bret King', 'Bret King']]
Miscategorized results: []

Example 3
Errors: []
Missing results: ['General Catalyst', 'MultiOn', 'Agnetic', 'IBM', 'Microsoft']
Extra results: ['Maryletticoyne']
Matching results: []
Miscategorized results: []

Example 4
Errors: []
Missing results: []
Extra results: []
Matching results: [['Ethan Kolasky', 'Ethan Kolasky'], ['Natasha Monroy', 'Natasha Monroy']]
Miscategorized results: []

Example 5
Errors: []
Missing results: []
Extra results

In [16]:
num_labels = 0
num_missing = 0
for example in evaluator.example_evals:
    num_labels += len(example.labels)
    num_missing += len(example.missingResults)
print(num_labels)
print(num_missing)
print(num_missing/num_labels)

18
5
0.2777777777777778


In [16]:
evaluator.link_results(9, [['Hughes Hubbard & Reed', 'Hughes Hubbard & Reed LLP']])

['Hughes Hubbard & Reed', 'Hughes Hubbard & Reed LLP']
